# crawling_dbpia

### 수집: 제목, 저자, 년도, KCI등재여부

### step 0. 준비

In [2]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

### step 1. 제목, 저자 크롤링

In [128]:
# 크롤링 할 검색어
keyword = input("크롤링할 검색어 입력: ")

크롤링할 검색어 입력: 교사 and 통합교육효능감


In [129]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"./chromedriver/chromedriver.exe")

In [130]:
# 사이트 주소
driver.get("https://www.dbpia.co.kr/search/topSearch?startCount=0&collection=ALL&range=A&searchField=ALL&sort=RANK&query=%s&srchOption=*&includeAr=false&searchOption=*"%keyword)
time.sleep(5)

In [88]:
# 년도 수동 입력

In [131]:
title_list = []
name_list = []
year_list = []
kci_list = []

for i in range(2,100):
    # 10 페이지 일 경우 next
    if i % 10 == 0:
        try:
            driver.find_element_by_css_selector("#next").click()
            time.sleep(5)
        except:
            print("next가 없음")
            break    
            
    # 제목 클롤링
    titles = driver.find_elements_by_css_selector(".titWrap")
    for tit in titles:
        title = tit.text
        title_list.append(title)
    
    infos = driver.find_elements_by_css_selector(".info")
    # 인포
    for inf in infos[0:-2]:
        info = inf.text
        info_spl = info.split('\n')
        
        # 저자 크롤링
        name = info_spl[0]
        name_list.append(name)
        
        # KCI 크롤링
        if 'KCI' in info_spl[-1]:
            kci_list.append(info_spl[-1])
        if 'KCI' not in info_spl[-1]:
            kci_list.append('미등재')

        # 날짜 크롤링
        for in_sp in info_spl:
            if '20' in in_sp and 'page' not in in_sp and '(' not in in_sp:
                year_list.append(in_sp[0:4])
        
    print(len(title_list), len(name_list), len(kci_list), len(year_list))
    
    # 일의 자리가 1이 아닐 경우 페이지 넘기기
    if i % 10 != 1:
        try:
            driver.find_element_by_css_selector("#pcPaging%s"%i).click()
            time.sleep(5)
        except:
            print("마지막 페이지")
            break
# driver.close()

20 20 20 20
40 40 40 40
60 60 60 60
80 80 80 80
100 100 100 100
102 102 102 102
마지막 페이지


In [132]:
df = pd.DataFrame({'저자':name_list, '연도':year_list, '논문제목':title_list, 'KCI 등재여부':kci_list})
df

크롤링 글 수 : 102


,저자,연도,논문제목,KCI 등재여부
0,"남숙, 곽승철",2020,통합교육 맥락에서 예비초등교사의 자기효능감 변화분석: 특수교육학개론 이수 전과 후,미등재
1,김성범,2020,통합상황에서 ADHD 아동을 교육하는 일반교사의 직무스트레스와 심리적소진의 관계에서...,미등재
2,"서효정, 박윤정",2018,"특수교육학개론 수업을 통한 예비 일반교사의 통합교육, 교수적 수정 실행, 교사 효능...",KCI등재
3,"지덕영, 맹희주, 손연아",2019,지속가능발전교육과 과학교육의 통합에 대한 초·중등 과학교사들의 인식,KCI등재
4,"김수진, 이미숙",2019,예비교사의 장애 및 통합교육 인식 변화를 위한 '문제중심학습(PBL)' 적용 효과 연구,KCI등재
...,...,...,...,...
97,"한정규, 이동형",2018,초등학생의 학급풍토 관련변인에 대한 메타분석,KCI등재
98,배옥현,2017,경상북도 다문화가족자녀 역량강화를 위한 지원방안,미등재
99,"박민애, 손원숙",2016,학습을 위한 평가 척도(AFL-Q)의 타당화,KCI등재
100,"차영숙, 최정아",2015,유아 음악수업의 컨설팅 매뉴얼 개발을 위한 기초연구,KCI등재


In [133]:
# 엑셀 저장
df.to_excel("dbpia_%s.xlsx"%keyword, encoding='utf-8-sig')